# Hands-on Game Of Life

---
**Requirements:**

- [Get started](./Get_started.ipynb)
- [Data management](./Data_management.ipynb)

---

The Game of Life is a cellular automaton developed by John Conway in 1970. In this "Game" a grid is filled with an __initial state__ of cells having either the status "dead" or "alive". From this initial state, several generations are computed and we can follow the evolution of the cells for each __generation__.

The rules are simple:

- For "dead" cells:
  - If it has exactly 3 neighbors the cell becomes __alive__ at the next generation
  
  <img alt="Cell birth" src="../../pictures/naissance.png" style="float:none"/>

- For "alive" cells:
  - If it has more than 3 neighbors the cell becomes __dead__ because of overpopulation 
  
  <img alt="Cell death overpopulation" src="../../pictures/mort_sur_pop.png" style="float:none"/>
  
  - If it has less than 2 neighbors the cell becomes __dead__ because of underpopulation 
  
  <img alt="Cell death underpopulation" src="../../pictures/mort_sous_pop.png" style="float:none"/>

For all other situations the state of the cell is kept unchanged. 

<img alt="No change" src="../../pictures/no_change.png" style="float:none"/>

## What to do

In this hands-on you have to add the directives to perform the following actions:

- Copy the initial state of the world generated on the CPU to the GPU
- Make sure that the computation of the current generation and the saving of the previous one occur on the GPU
- Compute the number of cells alive for the current generation is done on the GPU
- The memory on the GPU is allocated and freed when the arrays are not needed anymore

Example stored in: `../../examples/Fortran/GameOfLife_exercise.f90`

In [ ]:
%%idrrun  -a --cliopts "20000 1000 300" 
program gol
    implicit none
    integer :: rows, cols, generations
    integer, allocatable :: world(:,:), old_world(:,:)
    integer :: g
    integer :: long
    character(len=:), allocatable :: arg
    integer :: i
    integer, dimension(3) :: n
    ! Read cmd line args
    DO i=1, COMMAND_ARGUMENT_COUNT()
       CALL GET_COMMAND_ARGUMENT(NUMBER=i, LENGTH=long)
       ALLOCATE(CHARACTER(len=long) :: arg)
       CALL GET_COMMAND_ARGUMENT(NUMBER=i, VALUE=arg)
       READ(arg,'(i10)') n(i)
       DEALLOCATE(arg)
    END DO
    rows=n(1)
    cols=n(2)
    generations=n(3)
    allocate(world(0:rows+1,0:cols+1), old_world(0:rows+1,0:cols+1))
    old_world(:,:) = 0
    call fill_world
    do g=1,generations
        call save_world(rows,cols, world(:,:), old_world(:,:))
        call next(rows,cols, world(:,:), old_world(:,:))
        print *, "Cells alive at generation ", g, ": ", alive(rows, cols, world)
        call output_world("generation", g, world(:,:), rows, cols)
    enddo
    deallocate(world, old_world)
    contains
    integer function alive(rows, cols, world) result(cells)
        implicit none
        integer, intent(in) :: rows, cols
        integer, intent(in) :: world(0:rows+1,0:cols+1)
        integer :: r,c
        cells = 0
        do r=1, rows
            do c=1, cols
               cells = cells + world(r,c) 
            enddo
        enddo
        end function alive
    subroutine save_world(rows,cols,world, old_world)
        implicit none
        integer, intent(in) :: rows,cols
        integer, intent(in) :: world(0:rows+1,0:cols+1)
        integer, intent(out) :: old_world(0:rows+1,0:cols+1)
        integer :: r,c
        do r=1, rows
            do c=1, cols
                old_world(r,c) = world(r,c)
            enddo
        enddo
    end subroutine save_world

    subroutine next(rows,cols,world, old_world)
        implicit none
        integer, intent(in) :: rows,cols
        integer, intent(in) :: old_world(0:rows+1,0:cols+1)
        integer, intent(out) :: world(0:rows+1,0:cols+1)
        integer :: r,c
        integer :: neigh
        do r=1, rows
            do c=1, cols
        neigh = old_world(r-1,c-1)+old_world(r,c-1)+old_world(r+1,c-1)+&
                old_world(r-1,c)+old_world(r+1,c)+&
                old_world(r-1,c+1)+old_world(r,c+1)+old_world(r+1,c+1)
                if (old_world(r,c) == 1 .and. (neigh<2.or.neigh>3) ) then
                    world(r,c) = 0
                else if (neigh == 3) then
                    world(r,c) = 1
                endif
            enddo
        enddo
    end subroutine next

    subroutine fill_world
        implicit none
        integer :: r,c, temp
        real*8 :: test
        do r=1,rows
            do c=1,cols
                call random_number(test)
                temp = mod(floor(test*100),4)
                if (temp.eq.0) then 
                    temp = 1
                else
                    temp = 0
                endif
                world(r,c) = temp
            enddo
        enddo
    end subroutine fill_world
    
    subroutine output_world(name, g, world, rows, cols)
        implicit none
        character(len=*), intent(in) :: name
        character(len=1024) :: filename
        integer, intent(in) :: g
        integer, intent(in) :: rows,cols
        integer, intent(in) :: world(0:rows+1,0:cols+1)
        integer :: r,c
        integer :: my_unit
        integer(kind=1) :: output(0:rows+1,0:cols+1)
        write(filename,"(A,I5.5,A)") trim(name), g,".gray"
        print *, trim(filename)
        do r=0,rows+1
            do c=0,cols+1
                output(r,c) = mod(world(r,c),2)*127
            enddo
        enddo
        open(newunit=my_unit, file=filename, access="stream", form="unformatted")
        write(my_unit, pos=1) output
        close(my_unit)
    end subroutine output_world
end program gol

In [ ]:
rows = 2000
cols = 1000

from idrcomp import convert_pic
import matplotlib.pyplot as plt
import glob
from PIL import Image
from ipywidgets import interact

files = sorted(glob.glob("*.gray"))
images = [convert_pic(f, cols, rows, "L") for f in files]
print(images[0].size)
def view(i):
    crop = (155,65,360,270)
    plt.figure(figsize=(12,12))
    plt.imshow(images[i].crop(crop), cmap="Greys")
    plt.show()
interact(view, i=(0, len(images)-1))

## Solution

Example stored in: `../../examples/Fortran/GameOfLife_solution.f90`

In [ ]:
%%idrrun  -a --cliopts "20000 1000 300" 
program gol
    implicit none
    integer :: rows, cols, generations
    integer, allocatable :: world(:,:), old_world(:,:)
    integer :: g
    integer :: long
    character(len=:), allocatable :: arg
    integer :: i
    integer, dimension(3) :: n
    ! Read cmd line args
    DO i=1, COMMAND_ARGUMENT_COUNT()
       CALL GET_COMMAND_ARGUMENT(NUMBER=i, LENGTH=long)
       ALLOCATE(CHARACTER(len=long) :: arg)
       CALL GET_COMMAND_ARGUMENT(NUMBER=i, VALUE=arg)
       READ(arg,'(i10)') n(i)
       DEALLOCATE(arg)
    END DO
    rows=n(1)
    cols=n(2)
    generations=n(3)
    allocate(world(0:rows+1,0:cols+1), old_world(0:rows+1,0:cols+1))
    old_world(:,:) = 0
    call fill_world
    !$ACC enter data copyin(world, old_world)
    do g=1,generations
        call save_world(rows,cols, world(:,:), old_world(:,:))
        call next(rows,cols, world(:,:), old_world(:,:))
        print *, "Cells alive at generation ", g, ": ", alive(rows, cols, world)
        call output_world("generation", g, world(:,:), rows, cols)
    enddo
    !$ACC exit data delete(world, old_world)
    deallocate(world, old_world)
    contains
    integer function alive(rows, cols, world) result(cells)
        implicit none
        integer, intent(in) :: rows, cols
        integer, intent(in) :: world(0:rows+1,0:cols+1)
        integer :: r,c
        cells = 0
        !$ACC parallel loop reduction(+:cells)
        do r=1, rows
            do c=1, cols
               cells = cells + world(r,c) 
            enddo
        enddo
        end function alive
    subroutine save_world(rows,cols,world, old_world)
        implicit none
        integer, intent(in) :: rows,cols
        integer, intent(in) :: world(0:rows+1,0:cols+1)
        integer, intent(out) :: old_world(0:rows+1,0:cols+1)
        integer :: r,c
        !$ACC parallel loop
        do r=1, rows
            do c=1, cols
                old_world(r,c) = world(r,c)
            enddo
        enddo
    end subroutine save_world

    subroutine next(rows,cols,world, old_world)
        implicit none
        integer, intent(in) :: rows,cols
        integer, intent(in) :: old_world(0:rows+1,0:cols+1)
        integer, intent(out) :: world(0:rows+1,0:cols+1)
        integer :: r,c
        integer :: neigh
        !$ACC parallel loop
        do r=1, rows
            do c=1, cols
        neigh = old_world(r-1,c-1)+old_world(r,c-1)+old_world(r+1,c-1)+&
                old_world(r-1,c)+old_world(r+1,c)+&
                old_world(r-1,c+1)+old_world(r,c+1)+old_world(r+1,c+1)
                if (old_world(r,c) == 1 .and. (neigh<2.or.neigh>3) ) then
                    world(r,c) = 0
                else if (neigh == 3) then
                    world(r,c) = 1
                endif
            enddo
        enddo
    end subroutine next

    subroutine fill_world
        implicit none
        integer :: r,c, temp
        real*8 :: test
        do r=1,rows
            do c=1,cols
                call random_number(test)
                temp = mod(floor(test*100),4)
                if (temp.eq.0) then 
                    temp = 1
                else
                    temp = 0
                endif
                world(r,c) = temp
            enddo
        enddo
    end subroutine fill_world
    
    subroutine output_world(name, g, world, rows, cols)
        implicit none
        character(len=*), intent(in) :: name
        character(len=1024) :: filename
        integer, intent(in) :: g
        integer, intent(in) :: rows,cols
        integer, intent(in) :: world(0:rows+1,0:cols+1)
        integer :: r,c
        integer :: my_unit
        integer(kind=1) :: output(0:rows+1,0:cols+1)
        write(filename,"(A,I5.5,A)") trim(name), g,".gray"
        print *, trim(filename)
        !$ACC update self(world)
        do r=0,rows+1
            do c=0,cols+1
                output(r,c) = mod(world(r,c),2)*127
            enddo
        enddo
        open(newunit=my_unit, file=filename, access="stream", form="unformatted")
        write(my_unit, pos=1) output
        close(my_unit)
    end subroutine output_world
end program gol

In [ ]:
rows = 2000
cols = 1000

from idrcomp import convert_pic
import matplotlib.pyplot as plt
import glob
from PIL import Image
from ipywidgets import interact

files = sorted(glob.glob("*.gray"))
images = [convert_pic(f, cols, rows, "L") for f in files]
print(images[0].size)
def view(i):
    crop = (155,65,360,270)
    plt.figure(figsize=(12,12))
    plt.imshow(images[i].crop(crop), cmap="Greys")
    plt.show()
interact(view, i=(0, len(images)-1))